# Pandas分组操作（groupby）

## 📚 学习目标
本教程将深入学习pandas中最强大的功能之一：分组操作（groupby）。

### 本教程将学习：
1. **基本分组** - groupby()的基本用法
2. **聚合操作** - sum、mean、count等聚合函数
3. **多列分组** - 基于多列进行分组
4. **分组后筛选** - filter()方法
5. **分组后转换** - transform()方法
6. **分组后应用** - apply()方法

### 学习重点：
- 理解分组的概念和原理
- 掌握各种聚合函数
- 学会分组后的数据处理

## 📦 导入库并创建示例数据

In [ ]:
import pandas as pd
import numpy as np

# 创建示例销售数据
data = {
    '销售员': ['张三', '李四', '王五', '张三', '李四', '王五', '张三', '李四'],
    '产品': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C'],
    '销售额': [1000, 1500, 1200, 2000, 1800, 1600, 900, 1100],
    '数量': [10, 15, 12, 20, 18, 16, 9, 11],
    '月份': [1, 1, 1, 2, 2, 2, 3, 3]
}
df = pd.DataFrame(data)

print("示例销售数据：")
print(df)

---
# 第一部分：基本分组操作

## ⭐ groupby()的核心概念

`groupby()` 将数据按照某个或某些列的值进行分组，然后对每组数据进行操作。

### 1. 单列分组

In [ ]:
# 按'销售员'列分组
grouped = df.groupby('销售员')
print("分组对象：")
print(type(grouped))
print(f"\n分组数量: {grouped.ngroups}")
print(f"分组名称: {list(grouped.groups.keys())}")

# groupby()返回一个GroupBy对象，需要配合聚合函数使用

### 2. 查看分组内容

In [ ]:
# 查看每个分组的内容
for name, group in grouped:
    print(f"\n分组: {name}")
    print(group)
    if name == '张三':  # 只显示第一个分组
        break

### 3. 使用get_group()获取特定分组

In [ ]:
# 获取特定分组的数据
group_zhang = grouped.get_group('张三')
print("'张三'分组的数据：")
print(group_zhang)

### 4. 多列分组

In [ ]:
# 按多列分组
grouped_multi = df.groupby(['销售员', '产品'])
print(f"多列分组数量: {grouped_multi.ngroups}")
print(f"\n分组组合: {list(grouped_multi.groups.keys())[:3]}...")

# 多列分组会创建多级索引

---
# 第二部分：聚合操作

## ⭐ 分组后的统计计算

### 1. 基本聚合函数 - sum()

In [ ]:
# 按销售员分组，计算总销售额
result = df.groupby('销售员')['销售额'].sum()
print("每个销售员的总销售额：")
print(result)
print(f"\n类型: {type(result)}")  # Series类型

### 2. 基本聚合函数 - mean()

In [ ]:
# 计算平均销售额
result = df.groupby('销售员')['销售额'].mean()
print("每个销售员的平均销售额：")
print(result.round(2))

### 3. 基本聚合函数 - count()

In [ ]:
# 统计每个销售员的订单数量
result = df.groupby('销售员')['销售额'].count()
print("每个销售员的订单数量：")
print(result)

### 4. 多个聚合函数 - agg()

In [ ]:
# 同时计算多个统计量
result = df.groupby('销售员')['销售额'].agg(['sum', 'mean', 'count', 'max', 'min'])
print("多个统计量：")
print(result)

# agg()可以同时应用多个函数

### 5. 不同列应用不同函数

In [ ]:
# 对不同列应用不同的聚合函数
result = df.groupby('销售员').agg({
    '销售额': 'sum',
    '数量': 'mean',
    '产品': 'count'
})
print("不同列应用不同函数：")
print(result)

### 6. 自定义聚合函数

In [ ]:
# 使用自定义函数
def range_func(x):
    return x.max() - x.min()

result = df.groupby('销售员')['销售额'].agg(range_func)
print("自定义函数（极差）：")
print(result)

# 实际应用：计算自定义指标

---
# 第三部分：分组后筛选

## ⭐ 使用filter()筛选分组

### 1. filter() - 基于分组统计筛选

In [ ]:
# 筛选总销售额大于3000的销售员的所有记录
result = df.groupby('销售员').filter(lambda x: x['销售额'].sum() > 3000)
print("总销售额>3000的销售员的所有记录：")
print(result)

# filter()返回满足条件的整个分组的数据

### 2. filter() - 基于分组大小筛选

In [ ]:
# 筛选订单数量大于2的销售员
result = df.groupby('销售员').filter(lambda x: len(x) > 2)
print("订单数量>2的销售员：")
print(result)

---
# 第四部分：分组后转换

## ⭐ 使用transform()转换数据

### 1. transform() - 添加分组统计列

In [ ]:
# 为每行添加该销售员的总销售额
df['销售员总销售额'] = df.groupby('销售员')['销售额'].transform('sum')
print("添加分组统计列：")
print(df[['销售员', '销售额', '销售员总销售额']])

# transform()返回与原始数据相同长度的Series

### 2. transform() - 标准化数据

In [ ]:
# 计算每个销售员的销售额相对于该销售员平均值的比例
df['销售额比例'] = df.groupby('销售员')['销售额'].transform(
    lambda x: x / x.mean()
)
print("标准化后的销售额：")
print(df[['销售员', '销售额', '销售额比例']].round(2))

### 3. transform() vs agg()的区别

In [ ]:
# agg()返回聚合后的结果（行数减少）
result_agg = df.groupby('销售员')['销售额'].agg('sum')
print("agg()结果（聚合后）：")
print(result_agg)
print(f"行数: {len(result_agg)}")

# transform()返回与原始数据相同长度的结果
result_transform = df.groupby('销售员')['销售额'].transform('sum')
print("\ntransform()结果（保持原长度）：")
print(result_transform)
print(f"行数: {len(result_transform)}")

---
# 第五部分：分组后应用自定义函数

## ⭐ 使用apply()应用自定义函数

### 1. apply() - 基本用法

In [ ]:
# 对每个分组应用自定义函数
def group_stats(group):
    return pd.Series({
        '总销售额': group['销售额'].sum(),
        '平均数量': group['数量'].mean(),
        '订单数': len(group)
    })

result = df.groupby('销售员').apply(group_stats)
print("应用自定义函数：")
print(result)

### 2. apply() - 返回DataFrame

In [ ]:
# 对每个分组进行排序
def sort_by_sales(group):
    return group.sort_values('销售额', ascending=False)

result = df.groupby('销售员').apply(sort_by_sales)
print("每个分组内按销售额排序：")
print(result)

# apply()可以返回任意形状的数据

### 3. apply() - 返回标量值

In [ ]:
# 返回每个分组的最大值
result = df.groupby('销售员')['销售额'].apply(lambda x: x.max())
print("每个销售员的最大销售额：")
print(result)

# 等价于：df.groupby('销售员')['销售额'].max()

---
# 第六部分：实际应用场景

## ⭐ 分组操作的实际应用

### 1. 计算每个销售员的业绩排名

In [ ]:
# 计算每个销售员的总销售额并排名
sales_summary = df.groupby('销售员')['销售额'].sum().sort_values(ascending=False)
sales_summary = sales_summary.reset_index()
sales_summary['排名'] = range(1, len(sales_summary) + 1)

print("销售员业绩排名：")
print(sales_summary)

# 实际应用：员工绩效排名、部门业绩对比等

### 2. 计算每个产品的平均销售额

In [ ]:
# 按产品分组，计算平均销售额
product_avg = df.groupby('产品')['销售额'].mean()
print("每个产品的平均销售额：")
print(product_avg.round(2))

### 3. 多维度分组统计

In [ ]:
# 按销售员和产品分组，计算总销售额
result = df.groupby(['销售员', '产品'])['销售额'].sum()
print("按销售员和产品分组统计：")
print(result)

# 实际应用：多维度数据分析

### 4. 分组后计算占比

In [ ]:
# 计算每个销售员的销售额占总销售额的比例
total_sales = df['销售额'].sum()
sales_by_person = df.groupby('销售员')['销售额'].sum()
sales_ratio = (sales_by_person / total_sales * 100).round(2)

print("每个销售员的销售额占比（%）：")
print(sales_ratio)

---
# 总结

## 🎯 本教程重点回顾

### 1. 基本分组
- `groupby()`: 创建分组对象
- 单列分组和多列分组
- 查看分组内容

### 2. 聚合操作
- `sum()`, `mean()`, `count()`: 基本聚合
- `agg()`: 多个聚合函数
- 自定义聚合函数

### 3. 分组后筛选
- `filter()`: 基于分组统计筛选
- 返回满足条件的整个分组

### 4. 分组后转换
- `transform()`: 添加分组统计列
- 保持原始数据长度
- 用于标准化和归一化

### 5. 分组后应用
- `apply()`: 应用自定义函数
- 可以返回任意形状的数据
- 最灵活的方法

### 6. 实际应用
- 业绩排名
- 多维度统计
- 计算占比

## 📚 下一步学习

掌握了分组操作后，接下来可以学习：
- 数据透视表（pivot_table）
- 数据合并（merge、concat）
- 时间序列处理

---

**记住：groupby是pandas最强大的功能之一，掌握它能让数据分析事半功倍！**